In [1]:
# import numpy as np 
# import torch

# import argparse
# import os
# import sys
# import uuid
# from pydub import AudioSegment
# from moviepy.editor import VideoFileClip, AudioFileClip
# # from google.cloud import texttospeech
# # from google.cloud import translate_v2 as translate
# import whisper
# import spacy
# from tqdm import tqdm
# import tempfile
# import re
# from model import VideoDubber
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
# import scipy
# from transformers import VitsModel, AutoTokenizer
# import librosa

# video_dubber = VideoDubber()
# args = dict()
# args['input'] = r'/Users/ostywise/Documents/Video_dubbing/russian_trailer.mp4'
# args['source_language'] = 'russian'

# audio_file = video_dubber.extract_audio_from_video(args['input'])

# transcription = video_dubber.transcribe_audio(audio_file, args['source_language'].lower(), model_size='small')
# # transcription = dict()
# # transcription['text'] = "Друзья, если вы смотрите это видео, значит, я уже умер. Но я бы хотел поблагодарить каждого, каждого человека, который был со мной и шел этот нелегкий путь от момента диагностики, фактически до самого конца. Спасибо вам за вашу поддержку, за ваши улыбки, за ваши отзывы. Я очень тронут каждым отзывам, которые я прочитал в интернете. Говорят, что человек живет пока помнят его имя и живет его дело. Сомнестно с друзьями моими Мития Лишковским и Сергеем Карповым мы придумали грант имени Андрея Павленко. Этот грант будет финансировать медиапроекты, которые доносят правильную, правдивую и настоящую информацию об онкологических заболеваниях, чтобы облегчить путь и жизнь нашим онкологическим пациентам. С этой целью, на платформе Patreon мы объявляем сбор денежных средств. 50% из этой суммы будет идти на финансирование медиапроектов, 50% пойдет на финансирование моей семьи, поскольку с смертью семья лишилась единственного кормильца. Внизу под видео есть ссылка на этот ресурс. Можете перейти и оформить любой комфортный для вас ежемесячный платеж. Помогая этому проекту, вы будете делать не только важное социальное дело, но и помогать моей семье. Спасибо вам большое"


## Ideas for AudioAligner architecture


In [160]:
source_language = 'russian'
target_language = 'english'
target_lang = 'en'

temp_files = []
ducked_audio = AudioSegment.from_wav(audio_file)

merged_audio = AudioSegment.silent(duration=0)

sentences = []
sentence_ends = []
sentence = ""
sent_start = 0
print("Composing sentences")
for segment in tqdm(transcription["segments"]):
    if segment["text"].isupper():
        continue
    for i, word in enumerate(segment["words"]):
        if not video_dubber.ISWORD.search(word["word"]):
            continue
        word["word"] = video_dubber.ABBREVIATIONS.get(word["word"].strip(), word["word"])
        if word["word"].startswith("-"):
            sentence = sentence[:-1] + word["word"] + " "
        else:
            sentence += word["word"]

        if word["word"].endswith("."):
            sentences.append(sentence)
            sentence_ends.append(word["end"])
            sent_start = 0
            sentence = ""

sentence_starts = [np.float64(0)] + sentence_ends[:-1]

Composing sentences


100%|██████████| 23/23 [00:00<00:00, 92758.65it/s]


In [161]:
len(sentences)

13

In [103]:
test_text = video_dubber.translate_text(''.join(sentences[:7]), target_lang=target_lang)

In [162]:
print("Translating sentences")
translated_texts = []
for sentence in tqdm(sentences):
    translated_chunk = video_dubber.translate_text(sentence, target_lang=target_lang)
    if translated_chunk is None:
        raise Exception("Translation failed")
    translated_texts.append(translated_chunk)

Translating sentences


100%|██████████| 13/13 [00:52<00:00,  4.00s/it]


In [153]:
# print("Translating sentences")
# step = 3
# translated_texts = ''
# for idx in tqdm(range(0, len(sentences), step)):
#     if idx+step > len(sentences): 
#         chunk = ''.join(sentences[idx:])
#     else:
#         chunk = ''.join(sentences[idx:idx+step])
#     translated_chunk = video_dubber.translate_text(chunk, target_lang=target_lang)
#     if translated_chunk is None:
#         raise Exception("Translation failed")
#     translated_texts += translated_chunk

In [186]:
translated_texts

["en, friends, if you're watching this video, then i'm already dead.",
 'en. (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN) (EN (EN)',
 'en. Eskerrik asko zuen babesaz, zuen irribariei, zuen iritziei.',
 "en. I'm so touched by every review I read on the internet.",
 'en, "Man lives as long as he remembers his name and lives his business".',
 "enko. We've got a grant named after Andrei Pavlenko.",
 'en, aided by the World Health Organization. The grant will fund m

In [187]:
print("Creating translated audio track")
prev_end_time = 0
for i, translated_text in enumerate(tqdm(translated_texts)):
    translated_audio_file = video_dubber.create_audio_from_text(translated_text, target_language)
    if translated_audio_file is None:
        raise Exception("Audio creation failed")
    temp_files.append(translated_audio_file)
    translated_audio = AudioSegment.from_wav(translated_audio_file)

    # Apply "ducking" effect: reduce volume of original audio during translated sentence
    start_time = int(sentence_starts[i] * 1000)
    end_time = start_time + len(translated_audio)
    next_start_time = int(sentence_starts[i+1] * 1000) if i < len(translated_texts) - 1 else len(ducked_audio)
    ducked_segment = ducked_audio[start_time:end_time].apply_gain(-10)  # adjust volume reduction as needed

    fade_out_duration = min(500, max(1, start_time - prev_end_time))
    fade_in_duration = min(500, max(1, next_start_time  - end_time))
    prev_end_time = end_time
    # Apply fade in effect to the end of the audio before the ducked segment
    if start_time == 0:
        ducked_audio = ducked_segment +  ducked_audio[end_time:].fade_in(fade_in_duration)
    elif end_time == len(ducked_audio):
        ducked_audio = ducked_audio[:start_time].fade_out(fade_out_duration) + ducked_segment
    else:
        ducked_audio = ducked_audio[:start_time].fade_out(fade_out_duration) \
                        + ducked_segment +  ducked_audio[end_time:].fade_in(fade_in_duration)

    # Overlay the translated audio on top of the original audio
    ducked_audio = ducked_audio.overlay(translated_audio, position=start_time)

    original_duration = int(sentence_ends[i] * 1000)
    new_duration = len(translated_audio) + len(merged_audio)
    padding_duration = max(0, original_duration - new_duration)
    padding = AudioSegment.silent(duration=padding_duration)
    merged_audio += padding + translated_audio
merged_audio, ducked_audio

Creating translated audio track


  0%|          | 0/13 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  8%|▊         | 1/13 [00:01<00:20,  1.74s/it]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the env

(<pydub.audio_segment.AudioSegment at 0x33d539d90>,
 <pydub.audio_segment.AudioSegment at 0x11d9999d0>)

In [188]:
merged_audio

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [191]:
output_filename = os.path.splitext(args['input'])[0] + ".wav"
video_dubber.save_audio_to_file(merged_audio, output_filename)

Audio track with translation only saved to /Users/ostywise/Documents/Video_dubbing/russian_trailer.wav


In [197]:
video_dubber.replace_audio_in_video(args['input'], ducked_audio)

AttributeError: 'VideoDubber' object has no attribute 'replace_audio_in_video'